In [ ]:
# Dependencies:
#   - numpy
#   - pandas
#   - tensorflow
#   - stellargraph

import numpy as np  # Linear algebra routines;
import pandas as pd # Data-frame processing, CSV file I/O (e.g. pd.read_csv);

# Load the StellarGraph library;
from stellargraph import StellarGraph
from stellargraph.data import EdgeSplitter
from stellargraph.mapper import FullBatchLinkGenerator
from stellargraph.layer import GCN, LinkEmbedding
from stellargraph.utils import plot_history
from tensorflow import keras 

# Used to load files;
import os

In [ ]:
# Input data files are available in the "./input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory;
print(os.listdir("./input"))

# Any results you write to the current directory are saved as output;
print(os.getcwd())

In [ ]:
# A list of utility functions used below to manipulate/clean data and compute accuracy metrics;

# Preprocess and clean the PPI dataset;
def build_dataframe(input_data: pd.DataFrame, col_name: str, preserve_int_col_name=False) -> pd.DataFrame:
    """
    Given an input DataFrame and a column name, return a new DataFrame in which the column has been cleaned.
    Used to transform features and labels columns from "0;1;1;0" to [0, 1, 1, 0]
    """
    vertices_dict = []
    for i, row_i in input_data.iterrows():
        features = [int(float(x)) for x in row_i[f"{col_name}s"].split(";")]
        
        new_v = {"id": i}
        for j, f in enumerate(features):
            new_v[j if preserve_int_col_name else f"{col_name}_{j}"] = f
        vertices_dict += [new_v]
    res_df = pd.DataFrame(vertices_dict)
    return res_df.set_index("id")

# Compute the F1 score from 2 vectors of binary labels (0, 1), (ground truth and predictions).
# Taken as-is from old Keras source code, before it got removed;
def f1_score(y_true, y_pred) -> float:
    true_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_true * y_pred, 0, 1)))
    possible_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_true, 0, 1)))
    predicted_positives = keras.backend.sum(keras.backend.round(keras.backend.clip(y_pred, 0, 1)))
    precision = true_positives / (predicted_positives + keras.backend.epsilon())
    recall = true_positives / (possible_positives + keras.backend.epsilon())
    f1_val = 2 * (precision * recall) / (precision + recall + keras.backend.epsilon())
    return f1_val

In [ ]:
# Define graph name;
graph_name = "ppi"

# Read vertex data;
vertices_path = f"./input/{graph_name}_v.csv"
vertices = pd.read_csv(vertices_path, sep=",", index_col="id")
vertices.drop(["dataset"], axis=1, inplace=True)
vertices = build_dataframe(vertices, "feature")
vertices.head()

In [ ]:
# Read edges data (training data);
edge_path = f"./input/{graph_name}_e_train.csv"
edges = pd.read_csv(edge_path, sep=",", index_col=None)
edges.head()

In [ ]:
# Build input graph from vertices and edges (training data) dataframes;
ppi_graph = StellarGraph(
    nodes={"protein": vertices},
    edges={"interaction": edges},
    source_column="source",
    target_column="dest"
    )
print(ppi_graph.info())

# Multigraph: there can be multiple edges between vertices -> Hint: most likely noise, it does not make much sense to have multiple edges here;
# Vertex features: float32 -> Maybe it should be int/bool?

In [ ]:
# Split the input graph into a train and validation graphs 
edge_splitter_validation = EdgeSplitter(ppi_graph)

# Randomly sample a fraction p=0.1 of all positive links, and same number of negative links, from ppi_graph, 
# Obtain the reduced graph graph_valid with the sampled links removed
# edges_validation and validation are employed as ground truth for testing the model;
graph_validation, edges_validation, labels_validation = edge_splitter_validation.train_test_split(
    p=0.1, method="global", keep_connected=True, seed=42
)
print(graph_validation.info())

# "global" -> pick random pairs of vertices to create fake edges ("negative samples")
# "keep_connected" -> Don't remove edges that would disconnect graphs
# p=0.1 -> remove 10% of edges

# graph_validation -> new graph with 10% of edges removed, we use it to create the training set
# edges_validation -> list of edges used for testing. 50% are real, 50% are fake
# labels_validation -> labels that identify real/fake edges in edges_test

In [ ]:
# Build a dataframe to visualize the real/fake edges (not used in the training);
edges_validation_sources = [edges_validation[i][0] for i in range(0, len(edges_validation))]
edges_validation_dests = [edges_validation[i][1] for i in range(0, len(edges_validation))]
edges_validation_df = pd.DataFrame(list(zip(edges_validation_sources, edges_validation_dests, labels_validation)), columns=["source", "dest", "label"])
print(edges_validation_df)

In [ ]:
# Here we create a dataset for training, by creating again some fake edges;
# Note: by passing the original "ppi_graph" as well, we guarantee that real edges 
#   that are no longer present in "graph_validation" cannot be created as fake edges.
# This avoids teaching the model that a real edge should be classified as fake.
# Is this a leak? Probably not, but think about it (and test your hypotheses)!
edge_splitter_train = EdgeSplitter(graph_validation, ppi_graph)

# Same as before, create fake edges to use during the training;
graph_train, edges_train, labels_train = edge_splitter_train.train_test_split(
    p=0.1, method="global", keep_connected=True, seed=42
)

# Note: you might want to do this step again (at the very beginning) to create a test set not employed during training;

In [ ]:
# Build training dataframe for explicit visualization;
edges_train_sources = [edges_train[i][0] for i in range(0, len(edges_train))]
edges_train_dests = [edges_train[i][1] for i in range(0, len(edges_train))]
edges_train_df = pd.DataFrame(list(zip(edges_train_sources, edges_train_dests, labels_train)), columns=["source", "dest", "label"])
print(edges_train_df)

In [ ]:
# Create a generator on graph_train and make an iterator over the training links.
# This class creates the data fed to the model during the training.
# This is used during the training to extract from the graph all the required information (e.g. vertex features given an edge).
# "gcn" is used to specify how data are preprocessed. Depending on your model of choice, you might need a different option: 
#   https://stellargraph.readthedocs.io/en/stable/api.html#stellargraph.mapper.FullBatchLinkGenerator
train_gen = FullBatchLinkGenerator(graph_train, method="gcn")
train_flow = train_gen.flow(edges_train, labels_train)

# Same thing, create a LinkGenerator to be used for validation;
validation_gen = FullBatchLinkGenerator(graph_validation, method="gcn")
validation_flow = validation_gen.flow(edges_validation, labels_validation)

In [ ]:
# Create a simple GCN model.
# We specify the generator used in the training here!
gcn = GCN(
    layer_sizes=[16, 16], activations=["relu", "relu"], generator=train_gen, dropout=0.3
)

# Expose input and output tensors of the GCN model for link prediction;
x_inp, x_out = gcn.in_out_tensors()

# Create the final link embedding layer: 
# - it takes a pair of vertex embeddings produced by the GCN model, 
# - applies a binary operator (ip = inner product) to produce the corresponding link embedding,
# - passes it through a dense layer (if method!="ip") to obtain the wanted embedding dimension
prediction = LinkEmbedding(activation="relu", method="ip")(x_out)

# Reshape the predictions from (X, 1) to (X,) to match the shape of targets;
prediction = keras.layers.Reshape((-1,))(prediction)

# Stack the GCN and prediction layers into a Keras model;
model = keras.Model(inputs=x_inp, outputs=prediction)

# Compile the model, specifying the hyperparameters;
model.compile(
    optimizer=keras.optimizers.Adam(lr=0.005),
    loss=keras.losses.binary_crossentropy,               # We do binary classification;
    metrics=[keras.metrics.BinaryAccuracy(), f1_score],  # Also measure binary accuracy and F1 score;
)

In [ ]:
# Evaluate the initial (untrained) model on the train and validation sets, to get baseline scores;
init_train_metrics = model.evaluate(train_flow)
init_valid_metrics = model.evaluate(validation_flow)

print("\nTrain Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nValidation Set Metrics of the initial (untrained) model:")
for name, val in zip(model.metrics_names, init_valid_metrics):
    print("\t{}: {:0.4f}".format(name, val))

# If you get F1=0, it's likely that everything is predicted as false and Recall is 0;

In [ ]:
# Set the number of training epochs;
epochs = 100

# Train the model (here, we specify the validation set);
history = model.fit(
    train_flow, epochs=epochs, validation_data=validation_flow, verbose=2, shuffle=False
)

In [ ]:
# Plot the training history
plot_history(history)

# Accuracy oscillates, and train and validation are close -> This indicates a high learning rate, and an incomplete training (underfitting);

In [ ]:
# Evaluate the trained model on the train and validation sets: 
train_metrics = model.evaluate(train_flow)
validation_metrics = model.evaluate(validation_flow)

print("\nTrain Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, train_metrics):
    print("\t{}: {:0.4f}".format(name, val))

print("\nValidation Set Metrics of the trained model:")
for name, val in zip(model.metrics_names, validation_metrics):
    print("\t{}: {:0.4f}".format(name, val))

In [ ]:
# Read edges data (test data)
edge_path = f"./input/{graph_name}_e_test.csv"
edges_test = pd.read_csv(edge_path, sep=",", index_col=None)

# Delete duplicated edges, if they exist;
edges_test = edges_test.drop_duplicates().reset_index(drop=True)

# Extract numpy arrays for test examples;
examples_test = edges_test.to_numpy()

edges_test.head()

In [ ]:
# Create a generator on ppi_graph and make an iterator over the test links;
test_gen = FullBatchLinkGenerator(ppi_graph, method="gcn")
test_flow = test_gen.flow(examples_test)

# Compute the predictions for the edges in the test set;
y_pred = model.predict(test_flow)
y_pred = keras.backend.round(keras.backend.clip(y_pred, 0, 1)).numpy()

# Attach the predictions to the test dataframe, then save it as the final output file.
# The output file should be "id,label", where "id" is "source_dest";
y_pred_df = pd.DataFrame(np.transpose(y_pred).astype(int), columns=["label"])
output = pd.concat([edges_test, y_pred_df], axis=1)
output["id"] = output["source"].astype(str) + "_" + output["dest"].astype(str)
output[["id", "label"]].to_csv(f"sample_{graph_name}_predictions.csv", index=False)
print(output)